In [1]:
import pandas as pd
import numpy as np

In [2]:
file = 'training_sample_by_tweet.tsv'

In [3]:
column_names = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified",\
               "engaging_user_account_creation", "engaged_follows_engaging", "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]

In [4]:
df = pd.read_csv(file, header=None, names=column_names, delimiter='\x01')

FileNotFoundError: File b'../data/training_sample.tsv' does not exist

In [ ]:
pd.set_option('display.max_columns', None)
print(df.shape)
display(df.head())

Parse attributes containing tab-separated lists into lists. Transform raw timestamps into human-readable timestamps.

In [6]:
df['text_tokens'] = df['text_tokens'].str.split('\t')

def to_hex_list(x):
    output = str(x).split('\t')
#     output = [int(val, 16) for val in str(x).split('\t')] 
    return output

cols_to_process = ['hashtags', 'present_media', 'present_links', 'present_domains']

for col in cols_to_process:  
    df[col] = df[col].apply(lambda x: to_hex_list(x) if isinstance(x, str)  else x)


    
cols_to_process = ['tweet_timestamp', 'engaging_user_account_creation', 'reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']

for col in cols_to_process:  
    df[col] = df[col].apply(lambda x: pd.Timestamp(x, unit='s'))

In [7]:
display(df.head())

,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaged_with_user_account_creation,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engaged_follows_engaging,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,"[101, 56898, 137, 174, 63247, 10526, 131, 3197...",NaN,3C21DCFB8E3FEC1CB3D2BFB413A78220,[Video],NaN,NaN,Retweet,76B8A9C3013AE6414A3E6012413CDC3B,2020-02-12 00:28:43,D1AA2C85FA644D64346EDD88470525F2,737,706,False,1403069820,000046C8606F1C3F5A7296222C88084B,131,2105,False,2019-11-17 08:11:09,False,NaT,NaT,NaT,NaT
1,"[101, 102463, 10230, 10105, 21040, 10169, 1281...",NaN,3D87CC3655C276F1771752081423B405,NaN,[BB422AA00380E45F312FD2CAA75F4960],[92D397F8E0F1E77B36B8C612C2C51E23],TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,2020-02-06 07:49:51,4DC65AC7BD963DE1F7617C047C33DE99,52366425,2383,True,1230139136,00006047187D0D18598EF12A650E1DAC,22,50,False,2012-06-26 01:26:02,False,NaT,NaT,NaT,NaT
2,"[101, 56898, 137, 11255, 22037, 10263, 168, 11...",[DB32BD91C2F1B37BE700F374A07FBC61],3701848B96AA740528A2B0E247777D7D,NaN,[2423BA02A75DB2189335DDC3FB6B74A1],[6D323BE93766E79BE423FAC5C28BE39B],Retweet,22C448FF81263D4BAF2A176145EE9EAD,2020-02-09 14:07:12,5C671539CB41B9807E209349B101E9FF,988,167,False,1530094483,0000648BAA193AE4C625DDF789B57172,251,719,False,2016-02-26 08:01:11,False,NaT,NaT,NaT,NaT
3,"[101, 13073, 28757, 106, 100, 14120, 131, 120,...",NaN,18176C6AD2871729384062F073CCE94D,[Video],NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,2020-02-08 12:18:12,70B900BE17416923D1E236A38798F202,1228134,5413,False,1378699943,000071667F50BAFEA722A8E8284581E5,18,58,False,2013-09-06 00:32:44,False,NaT,NaT,NaT,2020-02-10 03:29:24
4,"[101, 3460, 1923, 6632, 2824, 30368, 2179, 188...",NaN,AF11AF01F842E7F120667B7B0B38676D,NaN,NaN,NaN,Quote,22C448FF81263D4BAF2A176145EE9EAD,2020-02-09 07:34:10,E94C0E9E8494F3D603F9D1A5C5242E3D,73,299,False,1549054499,00007745A6EE969F1A0F44B10DC17671,268,526,False,2009-09-07 03:40:00,False,NaT,NaT,NaT,NaT


## T4 - Extract the Social Network 

In [8]:
from scipy import sparse as sp

In [9]:
unique_user_ids = df['engaging_user_id'].append(df['engaged_with_user_id']).unique()

m = len(unique_user_ids)

userId_to_userIDX = dict(zip(unique_user_ids, range(m)))
userIDX_to_userId = dict(zip(range(m), unique_user_ids))

In [10]:
e_df = df[df['engaged_follows_engaging']][['engaging_user_id', 'engaged_with_user_id', 'engaged_follows_engaging']]

e_df['engaging_user_idx'] = e_df['engaging_user_id'].map(userId_to_userIDX)
e_df['engaged_with_user_idx'] = e_df['engaged_with_user_id'].map(userId_to_userIDX)

e_df.sort_values(by=['engaging_user_idx'], inplace = True)
e_df.drop_duplicates(inplace = True)
e_df.reset_index(drop = True, inplace = True)

display(e_df.head())

,engaging_user_id,engaged_with_user_id,engaged_follows_engaging,engaging_user_idx,engaged_with_user_idx
0,00006047187D0D18598EF12A650E1DAC,AA1F44078BF66B12F87E9E5064B9BEC6,True,1,55103
1,0000648BAA193AE4C625DDF789B57172,F1AE231029BFC328EE863FDE84883987,True,2,62418
2,00007745A6EE969F1A0F44B10DC17671,3C315728DF00E6FBF60E0542F4E5C62A,True,4,32559
3,00007745A6EE969F1A0F44B10DC17671,02FE74934CC2CEEEC4B821064FACC574,True,4,57017
4,00007745A6EE969F1A0F44B10DC17671,BF3EC3EBAF644CFAFF3627FF25EE0B27,True,4,20526


In [11]:
SN = sp.csr_matrix((e_df.engaged_follows_engaging, (e_df.engaging_user_idx, e_df.engaged_with_user_idx)), shape=(m, m))

In [12]:
SN.shape

(65963, 65963)